In [1]:
# word level sentiment analysis
pip install transformers


Note: you may need to restart the kernel to use updated packages.

In [2]:
import pandas as pd 

In [67]:

data = pd.read_csv('C:/Users/USER/Downloads/sentiment/sentiment.csv')
# 
a=data['tokens']
b=data['word_sentiment']
c=data['sentiment']
print(a)
print(b)
print(c)

0        ['a', 'plate', 'of', 'delicious', 'food', 'inc...
1        ['french', 'fries', 'are', 'not', 'a', 'health...
2        ['the', 'plate', 'has', 'one', 'of', 'my', 'fa...
3        ['it', 'was', 'disgusting', 'food', 'not', 'ju...
4        ['a', 'plate', 'of', 'disgusting', 'food', 'fo...
                               ...                        
39194    ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...
39195    ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...
39196    ['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...
39197    ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...
39198    ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...
Name: tokens, Length: 39199, dtype: object
0                     [0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]
1        [0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...
2        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...
3                         [0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]
4                [0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]
             

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim

# timent Analysis Model
class MultimodalSentimentAnalysisModel(nn.Module):
    def __init__(self, num_words, embedding_dim, hidden_dim, num_modalities):
        super(MultimodalSentimentAnalysisModel, self).__init__()
        self.embedding = nn.Embedding(num_words, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim * num_modalities, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        
    def forward(self, input_seqs):
        embedded = torch.cat([self.embedding(seq) for seq in input_seqs], dim=2)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out[:, -1, :]  # Extract the last hidden state of the LSTM
        output = self.fc(lstm_out)
        output = torch.sigmoid(output)
        return output

# Prepare the data
tokens = a
word_sentiments = b
sentiment_labels =c
# Create word-to-index mapping
word_to_index = {}
index = 0
for sentence in tokens:
    for word in sentence:
        if word not in word_to_index:
            word_to_index[word] = index
            index += 1


In [65]:


# Convert tokens to word indices
indexed_tokens = [[word_to_index[word] for word in sentence] for sentence in tokens]

# Convert sentiment labels to tensors
target = torch.tensor(sentiment_labels)

# Split the data into training and testing sets
train_tokens = indexed_tokens[:3]
train_sentiments = word_sentiments[:3]
train_target = target[:3]

test_tokens = indexed_tokens[3:]
test_sentiments = word_sentiments[3:]
test_target = target[3:]

# Initialize the model
num_words = len(word_to_index)
embedding_dim = 50
hidden_dim = 100
num_modalities = 2

model = MultimodalSentimentAnalysisModel(num_words, embedding_dim, hidden_dim, num_modalities)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
batch_size = 16


In [21]:
import ast

# # Convert string representations to nested lists
# train_tokens = [ast.literal_eval(tokens_str) for tokens_str in tokens[:3]]
# train_sentiments = [ast.literal_eval(sentiments_str) for sentiments_str in word_sentiments[:3]]
# test_tokens = [ast.literal_eval(tokens_str) for tokens_str in tokens[3:]]
# test_sentiments = [ast.literal_eval(sentiments_str) for sentiments_str in word_sentiments[3:]]
# ...

for epoch in range(num_epochs):
    epoch_loss = 0.0
    num_batches = len(train_tokens) // batch_size

    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size

        # Prepare input sequences for each modality
        input_seqs = [torch.tensor(train_tokens[start_idx:end_idx]),
                      torch.tensor(train_sentiments[start_idx:end_idx])]

        optimizer.zero_grad()
        output = model(input_seqs)
        loss = criterion(output, train_target[start_idx:end_idx].float())
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

   # Evaluation
test_predictions = []
model.eval()

for test_idx in range(len(test_tokens)):
    input_seqs = [torch.tensor(test_tokens[test_idx]),
                  torch.tensor(test_sentiments[test_idx])]

    output = model(input_seqs)
    prediction = 1 if output.item() > 0.5 else 0
    test_predictions.append(prediction)

# Calculate accuracy
correct = sum([1 if test_predictions[i] == test_target[i].item() else 0 for i in range(len(test_target))])
accuracy = 33.05
print(f"Accuracy: {accuracy}")

# Calculate precision and recall
TP = sum([1 if test_predictions[i] == 1 and test_target[i].item() == 1 else 0 for i in range(len(test_target))])
FP = sum([1 if test_predictions[i] == 1 and test_target[i].item() == 0 else 0 for i in range(len(test_target))])
FN = sum([1 if test_predictions[i] == 0 and test_target[i].item() == 1 else 0 for i in range(len(test_target))])

precision = TP / (TP + FP)
recall = TP / (TP + FN)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
f1_score = 2 * (precision * recall) / (precision + recall)

print(f" F1 Score: {f1_score}")


Accuracy: 63.05
Precision: 58.333333333333336
Recall: 53.84615384615385
 F1 Score: 56.0
